# Optimizing Financial Aid Resources

**Notebook by [Dev Nambi](https://blogs.uw.edu/fractals/)**


### Goals

This notebook uses the enrollment prediction model created elsewhere and optimizes the distribution of funds given the constraints involved (total scholarship funding, explainability).

* Run 


### Process

* A. Define constraints, such as the average SAT score, or average high school GPA, the different scholarship values, and the number of students to admit.
* B. Load in student data, software libraries, and model objects.
* C. Pass 1 - Arbitrary Predictions
* D. Calculate and visualize yield, net revenue, and scholarship info per segment.
* E. Pass 2 - Transfer Learning
* F. Calculate and visualize yield, net revenue, and scholarship info in an explainable way.
* G. Save off the results for later use.


### Resources

* https://github.com/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/pareto-optimized-road-trip/optimized-state-capitols-trip.ipynb

## Part A - Define Goals and Constraints

Next we're going to have to specify constraints we care about:

* **Maximum Student Enrollment** - The maximum number of domestic non-resident students you would like to enroll
* **Average Student SAT** - The minimum average SAT for domestic non-resident students you're willing to accept
* **Award Amounts** - The different scholarship amounts to award.
* **Scholarship Fund Amount** - The amount of money to distribute via Purple and Gold scholarships

#### First, Define Constraints

In [1]:
fund_amount = 3000000   #3 million dollars

maximum_student_enrollment = 2100

average_student_sat = 1090

tuition_amount = 34143

scholarship_amounts = [0,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
#scholarship_amounts = [0,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
#scholarship_amounts = [0,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
max_scholarship = max(scholarship_amounts)

number_of_buckets = 20

## Part B - Load in student data, software libraries, and model objects

### Part B1 - Load software libraries

In [4]:
# Load in software libraries
import os
import sys
import pandas as pd
import numpy as np
import pickle
import random
import copy
from tqdm import tqdm

# Get data-munging libraries
from sklearn import preprocessing
from sklearn import decomposition

# Get ML libraries
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# Get Viz libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Genetic algorithm
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

### Part B2 - Load  objects

In [2]:
model_location = 'logit_model.pkl'
scalar_location = 'model_scalar.pkl'

with open(model_location, 'rb') as model_input:
    predict_model = pickle.load(model_input)

with open(scalar_location, 'rb') as scalar_input:
    model_scalar = pickle.load(scalar_input)

#print (str(predict_model))
#print (str(model_scalar))

df = pd.read_csv('Transformed_Data_set.csv')

### Part C - First Pass, Arbitrary Predictions

1. Get the data into a handy format
2. Predict students who will come with no PuGo
3. For each scholarship amount, predict students who will switch and attend

**Reminder Constraints**

fund_amount
average_student_sat
award_amounts
maximum_student_enrollment

In [3]:
df_IsEnrolled = df.IsEnrolledOrMatriculated
df_2 = df.copy()
#df_2.drop('IsEnrolledOrMatriculated', axis=1, inplace=True)

In [4]:
df_unscaled = pd.DataFrame(model_scalar.inverse_transform(df_2), columns=df_2.columns)
#df_unscaled['IsEnrolledOrMatriculated'] = df_IsEnrolled
#df_unscaled.describe()

In [5]:
df_unscaled['PuGo_Amount'] = 0
df_unscaled['EnrollPrediction'] = 0
df_unscaled['EnrollPredictionTemp'] = 0
df_unscaled['OriginalTuition_After_Grants'] = df_unscaled['tuition_after_grants']

### Part C1 - Predict which students will attend with no PuGo scholarship

In [6]:
df_scaled = pd.DataFrame(model_scalar.transform(df_unscaled.ix[:,0:27]), columns=df_unscaled.columns[0:27])
df_scaled['IsEnrolledOrMatriculated'] = df_IsEnrolled
df_scaled.drop('IsEnrolledOrMatriculated', axis=1, inplace=True)
#df_scaled.columns

In [20]:
# Interaction terms
interactions_x = pd.DataFrame(preprocessing.PolynomialFeatures(2, interaction_only=True, include_bias=False).fit_transform(df_scaled))
# this produces a huge number of interaction terms. 

useful_interactions_x = interactions_x[useful_interaction_list]
#useful_interactions_x.columns
df_scaled = useful_interactions_x.join(df_scaled)
# Now describe the results
#df_scaled.describe()

In [21]:
# df_2.ix[:,0:31] <- use this to ignore columns added at the end
df_unscaled['EnrollPredictionTemp'] = predict_model.predict(df_scaled)

#df_unscaled.describe()

In [22]:
df_unscaled['EnrollPrediction'] = df_unscaled['EnrollPredictionTemp']

In [23]:
print ('Students who will enroll with no PuGo funding:',df_unscaled.EnrollPrediction.sum())

Students who will enroll with no PuGo funding: 1130


## Part D - Calculate and visualize yield, net revenue, and scholarship info per segment.

In [24]:
predicted_pugo_offered = df_unscaled.PuGo_Amount.sum()
print ('PuGo Offered',predicted_pugo_offered)

predicted_pugo_use = df_unscaled.PuGo_Amount[df_unscaled['EnrollPrediction'] == 1].sum()
print ('PuGo Use',predicted_pugo_use)

PuGo Offered 0
PuGo Use 0


In [25]:
predicted_student_count = df_unscaled.EnrollPrediction.sum()
total_student_count = df_unscaled.EnrollPrediction.count()

print ('Predicted students:',predicted_student_count)
print ('Admitted students:',total_student_count)
print ('Predicted yield:',predicted_student_count * 100.0 / total_student_count)

Predicted students: 1130
Admitted students: 15102
Predicted yield: 7.48245265528


In [26]:
predicted_net_revenue = df_unscaled.tuition_after_grants[df_unscaled['EnrollPrediction'] == 1].sum()
print ('Predicted Net Revenue:',predicted_net_revenue)

Predicted Net Revenue: 33883258.24337029


In [27]:
predicted_avg_sat = df_unscaled.TestScore[df_unscaled['EnrollPrediction'] == 1].mean()
print ('Predicted Average SAT:',predicted_avg_sat)

Predicted Average SAT: 1309.2831809858212


## Part E - Pass 2, Transfer Learning and Genetic Optimization

* Useful resources, https://github.com/DEAP/notebooks/blob/master/OneMax.ipynb
* https://github.com/DEAP/notebooks/blob/master/SIGEvolution.ipynb
* http://deap.readthedocs.io/en/master/tutorials/basic/part1.html

### E1 - Assign students to 'buckets' based on academic performance

* Assign each student to a bucket
* Analyze and report on the characteristics of each bucket

In [28]:
df_unscaled['BucketRank'] = df_unscaled['AcademicScore'].rank(ascending=1)

bucket_count = 20
rows_per_bucket = df_unscaled.BucketRank.count() / bucket_count

df_unscaled['Bucket'] = np.floor(df_unscaled['BucketRank'] / rows_per_bucket)

In [29]:
for i in range(0,20):
    high_school_gpa = df_unscaled.HighSchoolGPA[df_unscaled['Bucket'] == i].sum()
    min_high_school_gpa = df_unscaled.HighSchoolGPA[df_unscaled['Bucket'] == i].quantile(0.2)
    max_high_school_gpa = df_unscaled.HighSchoolGPA[df_unscaled['Bucket'] == i].quantile(0.8)
    test_score = df_unscaled.TestScore[df_unscaled['Bucket'] == i].sum()
    min_test_score = df_unscaled.TestScore[df_unscaled['Bucket'] == i].quantile(0.2)
    max_test_score = df_unscaled.TestScore[df_unscaled['Bucket'] == i].quantile(0.8)
    academic_score = df_unscaled.AcademicScore[df_unscaled['Bucket'] == i].sum()
    efc = df_unscaled.EFC[df_unscaled['Bucket'] == i].sum()
    students = df_unscaled.HighSchoolGPA[df_unscaled['Bucket'] == i].count()
    print ('Bucket',i,students, high_school_gpa*1.0/students, min_high_school_gpa, max_high_school_gpa, test_score*1.0/students, min_test_score, max_test_score, academic_score*1.0 / students, efc*1.0/students)



#df_unscaled[df_unscaled['Bucket'] == i].columns

Bucket 0 755 3.2642637468 3.1840680665393135 3.4296480243344174 1058.35784543 940.5456798240546 1284.903842066036 0.799349186138 27240.8961146
Bucket 1 755 3.52557871182 3.460345519058805 3.5708565000666015 1162.09952965 1077.345497701006 1341.510663256499 0.86239209638 39747.158076
Bucket 2 755 3.61250751145 3.5647170011217244 3.6445304874051327 1207.27252272 1115.083378494648 1379.248544050141 0.884026839022 42373.9353126
Bucket 3 752 3.6599200559 3.6199724916256226 3.693646478964154 1275.76419906 1171.6901996851107 1412.269189744577 0.898553460614 46340.1766173
Bucket 4 759 3.70020660234 3.669088483184643 3.730483472633419 1299.16120349 1199.9936102803422 1407.551954645372 0.908733002423 53228.1826381
Bucket 5 754 3.73365093296 3.699785977909031 3.7673204663026847 1313.9767751 1209.4280804787525 1412.2691897445773 0.91689717398 51162.8300099
Bucket 6 757 3.76360594779 3.7366229715782966 3.7918784620821953 1317.71884871 1218.862550677163 1412.269189744577 0.923617161738 45718.4474428

**Make a single function that returns revenue amounts**

The function returns:

   * net tuition revenue
   * scholarship amount
   * students enrolled
   * median test score
   * yield 

In [30]:
def parse_scholarships_base(buckets, include_bucket_counts):
    """
        This function returns the net tuition revenue, scholarship used, 
        student count, and average test score
    """
    tuition_revenue = 0
    scholarship_used = 0
    students = 0
    total_test_score = 0
    
    buckets = list(buckets)
    
    for i in range(len(buckets)):
        bucket_amount = buckets[i]
        #df_unscaled.PuGo_Amount[df_unscaled['Bucket'] == i] = bucket_amount
        #df_unscaled.PuGo_Amount[] = bucket_amount
        df_unscaled.ix[df_unscaled.Bucket == i, 'PuGo_Amount'] = bucket_amount
    
    df_unscaled['tuition_after_grants'] = df_unscaled['OriginalTuition_After_Grants'] - df_unscaled['PuGo_Amount']
    # Now scale back down    
    df_scaled = pd.DataFrame(model_scalar.transform(df_unscaled.ix[:,0:27]), columns=df_unscaled.columns[0:27])
    df_scaled.drop('IsEnrolledOrMatriculated', axis=1, inplace=True)
    
    # Now add interaction terms
    interactions_x = pd.DataFrame(preprocessing.PolynomialFeatures(2, interaction_only=True, include_bias=False).fit_transform(df_scaled))
    useful_interactions_x = interactions_x[useful_interaction_list]
    df_scaled = useful_interactions_x.join(df_scaled)
    
    # Now predict
    df_unscaled['EnrollPredictionTemp'] = predict_model.predict(df_scaled)
    
    students = df_unscaled.EnrollPredictionTemp[df_unscaled['EnrollPredictionTemp'] == 1].count()
    scholarship_used = df_unscaled.PuGo_Amount[df_unscaled['EnrollPredictionTemp'] == 1].sum()
    tuition_revenue = students * tuition_amount
    total_test_score = df_unscaled.TestScore[df_unscaled['EnrollPredictionTemp'] == 1].sum()
    total_students = df_unscaled.EnrollPredictionTemp.count()
    
    if include_bucket_counts:
        # look at the information per bucket
        for i in range(len(buckets)):
            bucket_students = df_unscaled.EnrollPredictionTemp[(df_unscaled['EnrollPredictionTemp'] == 1) & (df_unscaled['Bucket'] == i)].count()
            bucket_scholarship_used = df_unscaled.PuGo_Amount[(df_unscaled['EnrollPredictionTemp'] == 1) & (df_unscaled['Bucket'] == i)].sum()
            bucket_revenue = bucket_students * tuition_amount
            bucket_test_score = df_unscaled.TestScore[(df_unscaled['EnrollPredictionTemp'] == 1) & (df_unscaled['Bucket'] == i)].sum()
            bucket_total_students = df_unscaled.EnrollPredictionTemp[df_unscaled['Bucket'] == i].count()
            print ('Bucket ',i, bucket_students, bucket_scholarship_used, bucket_revenue - bucket_scholarship_used, bucket_test_score * 1.0 / bucket_students * 1.0, bucket_students * 1.0 / bucket_total_students)
        
    # return everything
    return tuition_revenue - scholarship_used, scholarship_used, students, total_test_score * 1.0 / students * 1.0, students * 1.0 / total_students

In [31]:
def parse_scholarships(buckets):
    """
        This function returns the net tuition revenue, scholarship used, 
        student count, and average test score
    """
    net_revenue, scholarship_used, students, avg_test_score, student_yield = parse_scholarships_base(buckets, False)
    return net_revenue, scholarship_used, students, avg_test_score, student_yield

In [32]:
buckets = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
net_revenue, scholarship_used, students, avg_test_score, student_yield = parse_scholarships(buckets)

print ('Net Revenue is ', net_revenue)
print ('Scholarship Used Is', scholarship_used)
print ('Student Count Is ',students)
print ('Average Test Score Is', avg_test_score)
print ('Student Yield Is',student_yield)

Net Revenue is  38581590
Scholarship Used Is 0
Student Count Is  1130
Average Test Score Is 1309.28318099
Student Yield Is 0.0748245265528


### Create the Evaluation Function

In [33]:
def eval_scholarships(bucket_combo):
    """
        This function returns the net tuition revenue as well as the number of students
        predicted to enroll.
        
        The genetic algorithm will favor scholarship distributinon that
        bring more money or more students.
    """
    net_revenue, scholarship_used, students, avg_test_score, student_yield = parse_scholarships(bucket_combo)
    
    net_revenue_to_return = net_revenue
    if scholarship_used > fund_amount:
        net_revenue_to_return = net_revenue_to_return * 1.0 * fund_amount / scholarship_used
    elif students > maximum_student_enrollment:
        net_revenue_to_return = net_revenue_to_return * 1.0 * maximum_student_enrollment / students 
    
    
    return net_revenue_to_return, avg_test_score

### Create the mutation function

In [143]:
np.random.choice([amt for amt in scholarship_amounts if amt > 10000],1)[0]

11000

In [198]:
def compare_lists(x, y):
    return 1 if len(frozenset(x).intersection(y)) == max(len(x),len(y)) else 0

In [199]:
a = [1,2,3,4,5]
b = [1,2,3,4,5]

compare_lists(a,b)

1

In [238]:
scholarship_amounts_2 = list(scholarship_amounts)

print (id(scholarship_amounts))
print (id(scholarship_amounts_2))
compare_lists(scholarship_amounts, scholarship_amounts_2)

199387656
203735304


1

In [276]:
def mutation_operator(bucket_combo):
    """
        This function applies a random change to one set of scholarships
        
            - Init: create a bucket with 20 '0' values if it's not the right size
            - Increase: Add a scholarship to a random bucket
            - Reduce: Reduce a scholarship in a random bucket
            - Swap: Swaps the scholarship amounts of two buckets
    """
    possible_mutations = ['swap']
    original_combo = list(bucket_combo)
    bucket_list = list(bucket_combo)
    itercount = 0
    print ('Bucketlist',bucket_list)
    print ('Originalcombo',original_combo)
    print ('Comparelists',compare_lists(original_combo, bucket_list))
    
    while compare_lists(original_combo, bucket_list) > 0:
        itercount = itercount + 1
        print ('Itercount',itercount)
        if np.min(bucket_combo) < max_scholarship:
            possible_mutations.append('increase')
        if np.max(bucket_combo) > 0:
            possible_mutations.append('reduce')
        #
        mutation_type = random.sample(possible_mutations, 1)[0]
        #print 'Mutation type: ',mutation_type
        #
        # Increase mutation
        if mutation_type == 'increase':
            to_increase = True
            while to_increase:
                index_to_increase = random.randint(0, number_of_buckets - 1)
                if bucket_combo[index_to_increase] < (max_scholarship):
                    bucket_combo[index_to_increase] = np.random.choice([amt for amt in scholarship_amounts if amt > bucket_combo[index_to_increase]],1)[0]
                    to_increase = False
        #
        # Reduce mutation
        elif mutation_type == 'reduce':
            index_to_delete = random.randint(0, number_of_buckets - 1)
            if bucket_combo[index_to_delete] > 0:
                bucket_combo[index_to_delete] = np.random.choice([amt for amt in scholarship_amounts if amt < bucket_combo[index_to_delete]],1)[0]
        #
        # Swap mutation
        elif mutation_type == 'swap':
            index1 = random.randint(0, number_of_buckets - 1)
            index2 = index1
            while index2 == index1:
                index2 = random.randint(0, number_of_buckets - 1)
                if bucket_combo[index1] == bucket_combo[index2]:
                    index2 = index1
            #
            bucket_combo[index1], bucket_combo[index2] = bucket_combo[index2], bucket_combo[index1]
        #
        bucket_combo.sort()
        bucket_list = list(bucket_combo)
    # now that things are sorted, return the result
    return bucket_combo,

### Create the initial attribute function

In [277]:
def random_scholarshipamount():
    return np.random.choice(scholarship_amounts, 1)[0]

In [278]:
def random_individual():
    random_buckets = np.random.choice(scholarship_amounts, size=number_of_buckets, replace=True).tolist()
    random_buckets.sort()
    return creator.Individual(random_buckets)

#### Type Creation

First step with DEAP is to create the required types. Usually the types created are the fitness and the individual. For the OneMax problem, we want to have a solution with as many ones as possible. Thus we need a maximizing fitness and a individual that is a sorted container, we'll choose the standard list.

Type creation is done by calling the function create in the creator module. This function takes two mandatory arguments and additional optional arguments. The first argument is the actual name of the type that we want to create. The second argument is the base classe that the new type created should inherit from. Finally, the optional arguments are members to add to the new type.

In [279]:
creator.create('FitnessMulti', base.Fitness, weights=(1.0, 0.001))
creator.create('Individual', list, fitness=creator.FitnessMulti)

In [280]:
all_buckets = range(0, bucket_count) #there are 20 buckets to start with
bucket_combinations = {}

buckets = [0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [281]:
ind = creator.Individual([1, 0, 1, 1, 0])

print(ind)
print(type(ind))
print(type(ind.fitness))
print(type(all_buckets))

[1, 0, 1, 1, 0]
<class 'deap.creator.Individual'>
<class 'deap.creator.FitnessMulti'>
<class 'range'>


#### Toolbox

The toolbox is intended to store functions with their arguments under standard aliases for uses in algorithms. Functions are registered by a function call with two mandatory arguments, the alias to give to the function and the function it will be associate with. Any additional argument will be given as argument when the alias is called.

In [282]:
toolbox = base.Toolbox()

This first one, attr_bool, calls randint from the random module with arguments (0, 1) to create an integer in the interval $[0, 1]$.

In [283]:
toolbox.register("attr_scholarshipamount", random_scholarshipamount)

In [284]:
bit = toolbox.attr_scholarshipamount()
print("bit is of type %s and has value\n%s" % (type(bit), bit))

bit is of type <class 'numpy.int32'> and has value
14000


The second function, individual, when called, will use the initRepeat function made available in the tools module to fill an Individual class with what is produced by 10 calls to the previously defined attr_scholarship function. 

* Note: this has been deprecated in favor of a custom function, random_individual, which returns sorted results

In [285]:
toolbox.register("individual", random_individual)

In [286]:
ind = toolbox.individual()
print("ind is of type %s and contains %d bits\n%s" % (type(ind), len(ind), ind))

ind is of type <class 'deap.creator.Individual'> and contains 20 bits
[0, 0, 3000, 4000, 4000, 4000, 6000, 6000, 6000, 7000, 7000, 7000, 7000, 7000, 8000, 9000, 10000, 11000, 12000, 13000]


The same thing is done for the population function.

In [287]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [288]:
pop = toolbox.population(n=3)
print("pop is of type %s and contains %d individuals\n%s" % (type(pop), len(pop), pop))

pop is of type <class 'list'> and contains 3 individuals
[[3000, 4000, 5000, 5000, 5000, 6000, 6000, 7000, 8000, 8000, 9000, 9000, 9000, 12000, 12000, 13000, 13000, 13000, 13000, 15000], [0, 0, 3000, 4000, 5000, 5000, 6000, 8000, 8000, 10000, 10000, 10000, 10000, 11000, 12000, 12000, 12000, 13000, 14000, 15000], [3000, 3000, 3000, 4000, 4000, 5000, 5000, 6000, 6000, 6000, 7000, 7000, 7000, 8000, 8000, 8000, 9000, 11000, 14000, 15000]]


## Genetic Operators

Registering the operators and their default arguments in the toolbox is done as follows:

The evaluation is given the alias evaluate. Having a single argument being the individual to evaluate we don’t need to fix any, the individual will be given later in the algorithm. 

In [289]:
toolbox.register('evaluate', eval_scholarships)

In [290]:
ind = toolbox.individual()
evaluation = toolbox.evaluate(ind)
print("Eval is of type %s and contains \n%s" % (type(evaluation), evaluation))

Eval is of type <class 'tuple'> and contains 
(8524929.328621909, 1342.4658059005485)


The two points crossover function is registered the same way under the alias mate. 

In [291]:
toolbox.register("mate", tools.cxTwoPoint)

The mutation, for its part, needs an argument to be fixed (the independent probability of each attribute to be mutated indpb). In the algorithms the mutate() function is called with the signature toolbox.mutate(mutant). This is the most convenient way because each mutation takes a different number of arguments, having those arguments fixed in the toolbox leave open most of the possibilities to change the mutation (crossover, selection, or evaluation) operator later in your researches.

In [292]:
toolbox.register('mutate', mutation_operator)

We can for example mutate an individual and expect 10% of its attributes to be flipped.

In [294]:
ind = toolbox.individual()
print(ind)
for i in range(0,2):
    toolbox.mutate(ind)
    revenue = toolbox.evaluate(ind)
    print('Revenue of ',revenue,'comes from',ind)

[0, 3000, 4000, 5000, 5000, 6000, 6000, 6000, 8000, 8000, 9000, 9000, 10000, 11000, 11000, 11000, 13000, 13000, 15000, 15000]
Bucketlist [0, 3000, 4000, 5000, 5000, 6000, 6000, 6000, 8000, 8000, 9000, 9000, 10000, 11000, 11000, 11000, 13000, 13000, 15000, 15000]
Originalcombo [0, 3000, 4000, 5000, 5000, 6000, 6000, 6000, 8000, 8000, 9000, 9000, 10000, 11000, 11000, 11000, 13000, 13000, 15000, 15000]
Comparelists 0
Revenue of  (8615942.8954423591, 1324.6862297978635) comes from [0, 3000, 4000, 5000, 5000, 6000, 6000, 6000, 8000, 8000, 9000, 9000, 10000, 11000, 11000, 11000, 13000, 13000, 15000, 15000]
Bucketlist [0, 3000, 4000, 5000, 5000, 6000, 6000, 6000, 8000, 8000, 9000, 9000, 10000, 11000, 11000, 11000, 13000, 13000, 15000, 15000]
Originalcombo [0, 3000, 4000, 5000, 5000, 6000, 6000, 6000, 8000, 8000, 9000, 9000, 10000, 11000, 11000, 11000, 13000, 13000, 15000, 15000]
Comparelists 0
Revenue of  (8615942.8954423591, 1324.6862297978635) comes from [0, 3000, 4000, 5000, 5000, 6000, 60

Finally, the selection operator is registered under the name select and the size of the tournament set to 3.

* https://pdfs.semanticscholar.org/2fb8/850864d71d6b9b30a6f4729a82cba456b027.pdf
* http://deap.readthedocs.io/en/master/api/tools.html

In [122]:
toolbox.register("select", tools.selTournament, tournsize=5)

### Evolving the Population

The main program shall be defined in a main function. All previous declarations are put in the global scope of the module to make created objects available for further use in other user experiments.

The main program is very simple. It consists of generating a population, and giving it to the algorithm for it to evolve a solution. Here we will employ some helpful introspection tools such as Statistics and a Hall of Fame. The statistics are computed using numpy functions on the population, and the hall of fame keeps track of the best individuals that ever appeared during the evolution. The algorithm take as arguments, among other, the population and the toolbox. Once the evolution is finished the population contains the individuals from the last generation.

In [54]:
pop = toolbox.population(n=300)

In [55]:
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("max", np.max)
stats.register("avg", np.mean)
stats.register("min", np.min)

In [56]:
fund_amount = 3000000   #3 million dollars

* http://deap.readthedocs.io/en/master/api/algo.html

In [57]:
pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.2, mutpb=0.7, ngen=40, stats=stats, halloffame=hof, verbose=True)

#cxpb 0.5 or 0? - The probability of mating two individuals.
#mutpb 0.2 or 1.0? - The probability of mutating an individual.

gen	nevals	max        	avg        	min    
0  	300   	1.08805e+07	3.79555e+06	1319.11
1  	227   	1.11429e+07	4.36946e+06	1318.52
2  	229   	1.20046e+07	4.84916e+06	1318.24
3  	234   	1.27468e+07	5.30434e+06	1317.75
4  	231   	1.37133e+07	5.65853e+06	1317.38
5  	232   	1.4687e+07 	6.02545e+06	1316.29
6  	219   	1.52253e+07	6.45381e+06	1316.29
7  	223   	1.7558e+07 	6.8706e+06 	1315.71
8  	228   	1.89931e+07	7.35731e+06	1315.7 
9  	227   	2.03779e+07	7.9328e+06 	1315.6 
10 	228   	2.06184e+07	8.68948e+06	1315.41
11 	233   	2.50349e+07	9.24679e+06	1315.36
12 	224   	2.91481e+07	9.84568e+06	1315.36
13 	234   	2.91481e+07	1.064e+07  	1315.03
14 	227   	3.38143e+07	1.17552e+07	1314.46
15 	238   	3.7275e+07 	1.31722e+07	1314.19
16 	232   	3.77506e+07	1.4785e+07 	1313.92
17 	227   	3.7761e+07 	1.63214e+07	1313.92
18 	230   	3.78395e+07	1.76557e+07	1312.64
19 	231   	3.78395e+07	1.81959e+07	1312.08
20 	224   	3.79056e+07	1.84393e+07	1311.48
21 	221   	3.79056e+07	1.84297e+07	1310.5 
22 	238   	

KeyboardInterrupt: 

In [ ]:
print("Best individual is: %s\nwith fitness: %s" % (hof[0], hof[0].fitness))

#print ('Evaluation of best individual:', parse_scholarships(hof[0]))
print ('Bucket, Students, ScholarshipUsed, NetRevenue, AvgTestScore, Yield')
print ('Evaluation of best individual detailed:', parse_scholarships_base((hof[0]), True))

import matplotlib.pyplot as plt
gen, avg, max_ = logbook.select("gen", "avg", "max")
plt.plot(gen, avg, label="average")
plt.plot(gen, max_, label="maximum")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend(loc="lower right")
plt.show()